In [1]:
import os, sys
import glob
import tqdm
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch

C:\Users\mustafa\miniforge3\envs\torchcv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Data exploration

In [2]:
sys.path.insert(0,'/Users/mustafa/Desktop/GitHub/SkinCancer/src')
from dataset import SkinCancer

In [3]:
df = pd.read_csv('../data/new_meta.csv')
df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,image_path
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,../data/HAM10k/HAM10000_images/ISIC_0027419.jpg
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,../data/HAM10k/HAM10000_images/ISIC_0025030.jpg
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,../data/HAM10k/HAM10000_images/ISIC_0026769.jpg
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,../data/HAM10k/HAM10000_images/ISIC_0025661.jpg
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,../data/HAM10k/HAM10000_images/ISIC_0031633.jpg


In [ ]:
counts = df['dx'].value_counts().to_dict()

In [ ]:
counts

In [ ]:
[1 - counts[i]/df.shape[0] for i in self.classes]

In [ ]:
df['image_path'][0]

In [4]:
root_path = '../data/HAM10k/HAM10000_images'


In [5]:
ds = SkinCancer(root_path,'../data/train.csv')

In [ ]:
ds.classes_all

In [ ]:
import torch
from sklearn.utils import class_weight

In [ ]:
y = df['dx'].to_numpy()

In [ ]:
del class_weights

In [ ]:
class_weights=class_weight.compute_class_weight('balanced',classes=np.unique(ds.classes),y=np.array(ds.classes_all))
class_weights=torch.tensor(class_weights,dtype=torch.float)

In [ ]:
class_weights.shape

In [ ]:
torch.tensor(ds.class_weights)

In [ ]:
for i,v in ds.class_count.items():
    w = 1 - v/ds.__len__()
    print(i, v, w)

In [ ]:
img , lable = ds.__getitem__(2)

In [ ]:
img

In [ ]:
dataloader = torch.utils.data.DataLoader(ds, batch_size=8,shuffle=True)

In [ ]:
next(iter(dataloader))

In [ ]:
ds.classes

In [6]:
from models import *

In [7]:
model = efficientnet()

In [8]:
model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [ ]:
model.classifier

In [ ]:
old_fc = model.classifier.__getitem__(-1)
new_fc = nn.Linear(in_features=old_fc.in_features, out_features= 7, bias=True)
model.classifier.__setitem__(-1 , new_fc)

In [ ]:
old_fc.in_features

In [ ]:
model.classifier

In [9]:
model2 = vit()

In [ ]:
model2.heads

In [ ]:
old_fc2 = model2.heads.__getitem__(-1)
new_fc2 = nn.Linear(in_features=old_fc2.in_features, out_features= 7, bias=True)
model2.heads.__setitem__(-1 , new_fc2)

In [10]:
model2.heads

Sequential(
  (head): Linear(in_features=768, out_features=7, bias=True)
)